<a href="https://colab.research.google.com/github/Kh0uloud/Optimization-for-City-Public-Transport-Route/blob/main/Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geojson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os, sys, requests, zipfile
import pandas as pd
from geojson import LineString, Feature, FeatureCollection, dump

def min2str(minutes):
    return "{:02d}:{:02d}".format(int((int(minutes) / 60) % 24), int(minutes % 60))

def str2min(time_str):
    h, m, s = time_str.strip().split(":")
    return int(h) * 60 + int(m)

In [ ]:
def zero_pad (x):
  if len(str(x))==1 :
    return('0'+str(x))
  else :
    return str(x)

In [ ]:
!mkdir /content/crowdedness

In [ ]:
def get_crowdedness_operator(operatorcode, type):

    for i in range(7, 21):
        url = "https://data.ndovloket.nl/bezetting/" + operatorcode.lower() + "/OC_" + \
              operatorcode.upper() + "_202306" + zero_pad(i) + "." + type
        filename = url.split("/")[-1]
        if not os.path.exists(os.path.join('crowdedness',filename)):
            with open(os.path.join('crowdedness',filename), "wb") as f:
                r = requests.get(url)
                f.write(r.content)

    return pd.read_csv(os.path.join('crowdedness', filename), low_memory=False,
                          compression='gzip' if type == 'csv.gz' else 'zip')

druktedata = get_crowdedness_operator('ns', 'csv.gz')

druktedata = druktedata[['DataOwnerCode', 'JourneyNumber', 'OperatingDay', 'UserStopCodeBegin',
                         'UserStopCodeEnd', 'VehicleType', 'TotalNumberOfCoaches', 'Occupancy']]
druktedata.columns=['operator', 'ritnumber', 'date', 'departure', 'arrival', 'wagontype',
                    'coaches', 'classification']
druktedata = druktedata[druktedata.departure.str.len() < 6] # Filter station abbrev

# Use an estimation of capacity per wagon based on wikipedia, just as rough estimation
druktedata = druktedata.merge(pd.DataFrame({'wagontype' : ['ICM', 'VIRM', 'SW7-25KV', 'SW9-25KV',
                                                           'SNG', 'SLT', 'FLIRT FFF', 'SGMM',
                                                           'DDZ'],
                                            'seats' : [60, 100, 50, 50, 85, 60, 55, 45, 50]}))
druktedata['seats'] = druktedata['seats'] * druktedata['coaches']

In [ ]:
druktedata

,operator,ritnumber,date,departure,arrival,wagontype,coaches,classification,seats
0,NS,2243,2023-06-20,SDM,DT,VIRM,4,3,400
1,NS,2237,2023-06-24,RSD,DDR,VIRM,4,2,400
2,NS,2465,2023-06-22,SDM,RTD,VIRM,10,2,1000
3,NS,2221,2023-06-20,HAD,HLM,VIRM,10,2,1000
4,NS,2442,2023-06-22,DDR,RTB,VIRM,4,2,400
...,...,...,...,...,...,...,...,...,...
316299,NS,911,2023-06-23,SHL,ASD,SW7-25KV,9,2,450
316300,NS,1005,2023-06-28,SHL,ASD,SW7-25KV,9,2,450
316301,NS,1039,2023-06-21,RTD,SHL,SW7-25KV,9,2,450
316302,NS,1172,2023-06-29,EHV,TB,SW7-25KV,9,2,450


In [ ]:
druktedata['date'].unique()

array(['2023-06-20', '2023-06-24', '2023-06-22', '2023-06-27',
       '2023-06-28', '2023-06-25', '2023-06-23', '2023-06-29',
       '2023-06-21', '2023-06-26'], dtype=object)

In [ ]:
druktedata[druktedata.date == '2023-06-20']

,operator,ritnumber,date,departure,arrival,wagontype,coaches,classification,seats
0,NS,2243,2023-06-20,SDM,DT,VIRM,4,3,400
3,NS,2221,2023-06-20,HAD,HLM,VIRM,10,2,1000
12,NS,3516,2023-06-20,HT,UT,VIRM,6,2,600
22,NS,3940,2023-06-20,BKG,BKF,VIRM,6,2,600
28,NS,2968,2023-06-20,ASB,ASA,VIRM,4,3,400
...,...,...,...,...,...,...,...,...,...
316265,NS,1034,2023-06-20,SHL,RTD,SW7-25KV,9,2,450
316267,NS,1005,2023-06-20,SHL,ASD,SW7-25KV,9,2,450
316282,NS,1037,2023-06-20,SHL,ASD,SW7-25KV,9,2,450
316283,NS,911,2023-06-20,SHL,ASD,SW7-25KV,9,3,450


In [ ]:
!mkdir /content/GTFS

In [ ]:
if not os.path.exists(os.path.join('GTFS','gtfs-nl.zip')):
    url = 'http://gtfs.ovapi.nl/nl/gtfs-nl.zip'
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(os.path.join('GTFS','gtfs-nl.zip'), 'wb') as f:
            for chunk in r.iter_content(chunk_size=1073741824):
                f.write(chunk)

In [ ]:
def read_csv(csvfile):
    with zipfile.ZipFile(os.path.join('GTFS','gtfs-nl.zip')) as z:
        with z.open(csvfile) as f:
            return pd.read_csv(f)

AGENCYNAMES = ['NS']
agencies = read_csv('agency.txt')
agency_ids = agencies[agencies.agency_name.isin(AGENCYNAMES)]['agency_id'].values
agencies = agencies[agencies.agency_name.isin(AGENCYNAMES)][['agency_id', 'agency_name']]

routes = read_csv('routes.txt')
routes = routes[routes.agency_id.isin(agency_ids)]
route_ids = routes.route_id.values
routes = routes[['route_id', 'agency_id', 'route_short_name', 'route_long_name', 'route_type']]

trips = read_csv('trips.txt')
trips = trips[trips.route_id.isin(route_ids)]
trip_ids = trips.trip_id.values
service_ids = trips.service_id.values
trips = trips[['route_id', 'service_id', 'trip_id', 'trip_headsign', 'trip_short_name', 'trip_long_name', 'direction_id', 'shape_id']]
trips.trip_short_name = trips.trip_short_name.astype(int)
trips.shape_id = trips.shape_id.astype('Int64')

calendar = read_csv('calendar_dates.txt')
calendar[calendar.service_id.isin(service_ids)]
calendar.date = calendar.date.astype(str)

trips = trips.merge(calendar[['service_id', 'date']], on='service_id')

stoptimes = read_csv('stop_times.txt')
stoptimes = stoptimes[stoptimes.trip_id.isin(trip_ids)]
stoptimes.stop_id = stoptimes.stop_id.astype(str)
stop_ids = stoptimes.stop_id.unique()
stoptimes = stoptimes[['trip_id', 'stop_sequence', 'stop_id', 'arrival_time',
                       'departure_time', 'shape_dist_traveled']]
stoptimes.arrival_time = stoptimes.arrival_time.apply(lambda x: str2min(x))
stoptimes.departure_time = stoptimes.departure_time.apply(lambda x: str2min(x))

# First get the stops (platforms)
stops_full = read_csv('stops.txt')
stops_full.stop_id = stops_full.stop_id.astype(str)
stops = stops_full[stops_full.stop_id.isin(stop_ids)].copy()

# Now add the stopareas (stations)
stopareas = stops.parent_station.unique()
stops = stops.append(stops_full[stops_full.stop_id.isin(stopareas)].copy())

stops.zone_id = stops.zone_id.str.replace('IFF:', '').str.upper()
stops.stop_code = stops.stop_code.str.upper()
stops = stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon', 'parent_station',
               'platform_code', 'stop_code', 'zone_id']]

stops.loc[stops['zone_id'].isnull(),'zone_id'] = stops['stop_code']
stops.loc[stops['stop_code'].isnull(),'stop_code'] = stops['zone_id']

# shapes = pd.read_csv(os.path.join('GTFS', 'shapes.txt'))
shapes = read_csv('shapes.txt')

<ipython-input-11-e1733569a074>:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f)
<ipython-input-11-e1733569a074>:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stops = stops.append(stops_full[stops_full.stop_id.isin(stopareas)].copy())


In [ ]:
stops

,stop_id,stop_name,stop_lat,stop_lon,parent_station,platform_code,stop_code,zone_id
13,2510280,Bodegraven,52.081613,4.745836,stoparea:17820,1,BDG,BDG
14,2510899,Gouda,52.017612,4.705700,stoparea:17806,10,GD,GD
18,2510177,Amsterdam Centraal,52.378765,4.900467,stoparea:18188,5,ASD,ASD
19,2511187,Hoorn Kersenboogerd,52.653364,5.084830,stoparea:17945,1,HNK,HNK
28,2511657,Nijkerk,52.222169,5.493628,stoparea:18260,1,NKK,NKK
...,...,...,...,...,...,...,...,...
53280,stoparea:153157,Eindhoven Strijp-S,51.450146,5.457029,NaN,NaN,EHS,EHS
53472,stoparea:194756,Zaandijk Zaanse Schans,52.469162,4.805161,NaN,NaN,ZZS,ZZS
53473,stoparea:194757,Koog aan de Zaan,52.457621,4.805914,NaN,NaN,KZ,KZ
63030,stoparea:400373,Rotterdam Beurs,51.918400,4.481146,NaN,NaN,BEURS,BEURS


Starting with the agencies, step by step, everything is filtered data for the specified operator. First agencies, then routes for the specified agencies, trips for these routes and stoptimes for these trips. Note that stops in GTFS are platforms in real life and stop- areas represent stations. <br> <br>
determine the location of the train for all trips of the day, between arrival at the first station and departure from the last station <br>
The route of the trip is specified as a GTFS shape consisting of a sequence of LatLon coordinates with the traveled distance from the first station <br>


In [ ]:
def interpolate_lat_lon(tripshape, dist):
    #tripshape: set of points from the GTFS shapefile specifying the route of the trip
    #dist: distance travelled along the tripshape
    ps = tripshape[tripshape['shape_dist_traveled'].le(dist)].index[-1]
    ns = tripshape[tripshape['shape_dist_traveled'].ge(dist)].index[0]
    lat1 = tripshape[tripshape.index == ps].iloc[0]['shape_pt_lat']
    lon1 = tripshape[tripshape.index == ps].iloc[0]['shape_pt_lon']
    lat2 = tripshape[tripshape.index == ns].iloc[0]['shape_pt_lat']
    lon2 = tripshape[tripshape.index == ns].iloc[0]['shape_pt_lon']
    dst1 = tripshape[tripshape.index == ps].iloc[0]['shape_dist_traveled']
    dst2 = tripshape[tripshape.index == ns].iloc[0]['shape_dist_traveled']
    prc = ((dist - dst1) / (dst2 - dst1)) if dst2 != dst1 else 1
    lat = lat1 + prc * (lat2 - lat1)
    lon = lon1 + prc * (lon2 - lon1)

    return lat, lon

In [ ]:
trips

,route_id,service_id,trip_id,trip_headsign,trip_short_name,trip_long_name,direction_id,shape_id,date
0,41238,2887,169353589,Dordrecht,2427,Intercity,0,1076786,20230630
1,41238,2887,169353589,Dordrecht,2427,Intercity,0,1076786,20230707
2,41238,2887,169353589,Dordrecht,2427,Intercity,0,1076786,20230714
3,53898,2887,169353590,Dordrecht,3516,Intercity,1,1081652,20230630
4,53898,2887,169353590,Dordrecht,3516,Intercity,1,1081652,20230707
...,...,...,...,...,...,...,...,...,...
572905,40775,1660,161766645,Nijmegen,900542,Snelbus i.p.v. trein,0,<NA>,20231109
572906,40775,1660,161766645,Nijmegen,900542,Snelbus i.p.v. trein,0,<NA>,20231116
572907,40775,1660,161766645,Nijmegen,900542,Snelbus i.p.v. trein,0,<NA>,20231123
572908,40775,1660,161766645,Nijmegen,900542,Snelbus i.p.v. trein,0,<NA>,20231130


In [ ]:
def get_trip_data(number, date):

  #return: dataframe with:
  #             LatLon location of this trip for every minute
  #             information on the last stoplocation the train visited

  trip = trips[(trips.trip_short_name==number) & (trips.date==date)].iloc[0]
  tripid = trip['trip_id']
  shapeid = trip['shape_id']
  tripshape = shapes[shapes.shape_id == shapeid]
  tripstops = stoptimes[stoptimes.trip_id==tripid].sort_values('stop_sequence')

  lastdep = -1
  lastdist= -1
  seq = 0
  dl = []

  for position in tripstops.iterrows():
    arr = position[1][3]
    dep = position[1][4]
    dist = position[1][5]
    dist = dist + 0.1 if dist == lastdist else dist
    arr = arr + 1 if arr == lastdep else arr
    stop = position[1][2]
    if (dist == dist) & (arr == arr):
      if lastdep > 0:
        seq = seq+1
        stepsize = ((dist-lastdist) / (arr-lastdep))
        for t in range(lastdep+1, arr):
          dst = int(lastdist + ( (t - lastdep) * stepsize))
          lat, lon = interpolate_lat_lon(tripshape, dst)
          dl.append({'trip_id': tripid, 'time': int(t), 'lat': lat, 'lon': lon,
                     'stop_id': stop, 'ritnumber' : number, 'sequence' : seq})
      seq = seq+1
      for t in range(arr, dep+1):
        dst = int(dist)
        lat, lon = interpolate_lat_lon(tripshape, dst)
        dl.append({'trip_id': tripid, 'time': int(t), 'lat': lat, 'lon': lon,
                   'stop_id': stop, 'ritnumber' : number, 'sequence' : seq})
      lastdep = dep
      lastdist = dist
  df = pd.DataFrame(dl)
  print(df.columns)
  #df.trip_id = df.trip_id.astype(int)
  #df.sequence = df.sequence.astype(int)
  #df.time = df.time.astype(int)
  #df.ritnumber = df.ritnumber.astype(int)
  #df.stop_id = df.stop_id.astype(int).astype(str)
  #df = df[['trip_id', 'ritnumber', 'sequence', 'time', 'stop_id', 'lat', 'lon']]

  return df

In [ ]:
trips[(trips.trip_short_name==2643) & (trips.date=='20230620')]

,route_id,service_id,trip_id,trip_headsign,trip_short_name,trip_long_name,direction_id,shape_id,date
547959,44942,1135,168243850,Almere Centrum,2643,Intercity,0,1060133,20230620


In [ ]:
trips

,route_id,service_id,trip_id,trip_headsign,trip_short_name,trip_long_name,direction_id,shape_id,date
0,41238,2887,169353589,Dordrecht,2427,Intercity,0,1076786,20230630
1,41238,2887,169353589,Dordrecht,2427,Intercity,0,1076786,20230707
2,41238,2887,169353589,Dordrecht,2427,Intercity,0,1076786,20230714
3,53898,2887,169353590,Dordrecht,3516,Intercity,1,1081652,20230630
4,53898,2887,169353590,Dordrecht,3516,Intercity,1,1081652,20230707
...,...,...,...,...,...,...,...,...,...
572905,40775,1660,161766645,Nijmegen,900542,Snelbus i.p.v. trein,0,<NA>,20231109
572906,40775,1660,161766645,Nijmegen,900542,Snelbus i.p.v. trein,0,<NA>,20231116
572907,40775,1660,161766645,Nijmegen,900542,Snelbus i.p.v. trein,0,<NA>,20231123
572908,40775,1660,161766645,Nijmegen,900542,Snelbus i.p.v. trein,0,<NA>,20231130


In [ ]:
trips[trips.date == '20230620']  #.trip_short_name.unique()[0]

,route_id,service_id,trip_id,trip_headsign,trip_short_name,trip_long_name,direction_id,shape_id,date
18499,17748,618,168758612,Uitgeest,4016,Sprinter,1,1068895,20230620
18503,83140,618,169070623,Dordrecht,6659,Sprinter,0,1081740,20230620
18507,73489,618,168772250,Tiel,6943,Sprinter,0,1034106,20230620
18511,73489,618,168772756,Tiel,6989,Sprinter,0,1034106,20230620
18515,84138,618,161742739,Amsterdam Centraal,919,Intercity direct,0,1078202,20230620
...,...,...,...,...,...,...,...,...,...
571219,41236,442,162699655,Wijchen,7670,Sprinter,1,1034116,20230620
572165,17601,685,162952053,Amersfoort Schothorst,11785,Intercity,0,1033836,20230620
572332,17601,1266,161761263,Utrecht Centraal,11788,Intercity,1,1033837,20230620
572409,67400,504,161761292,Eindhoven Centraal,13578,Intercity,1,1078206,20230620


In [ ]:
timedata = pd.DataFrame()
for r in trips[trips.date == '20230620'].trip_short_name.unique():
  timedata = timedata.append(get_trip_data(r, '20230620'))
timedata.trip_id = timedata.trip_id.astype(int)
timedata.sequence = timedata.sequence.astype(int)
timedata.time = timedata.time.astype(int)
timedata.ritnumber = timedata.ritnumber.astype(int)
timedata.stop_id = timedata.stop_id.astype(int).astype(str)
timedata = timedata[['trip_id', 'ritnumber', 'sequence', 'time', 'stop_id', 'lat', 'lon']]
timedata = timedata.merge(stops[['stop_id', 'stop_code', 'stop_name']])

Index(['trip_id', 'time', 'lat', 'lon', 'stop_id', 'ritnumber', 'sequence'], dtype='object')


<ipython-input-19-cb0d1c70705c>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  timedata = timedata.append(get_trip_data(r, '20230620'))


Index(['trip_id', 'time', 'lat', 'lon', 'stop_id', 'ritnumber', 'sequence'], dtype='object')


<ipython-input-19-cb0d1c70705c>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  timedata = timedata.append(get_trip_data(r, '20230620'))


Index(['trip_id', 'time', 'lat', 'lon', 'stop_id', 'ritnumber', 'sequence'], dtype='object')


...

<ipython-input-19-cb0d1c70705c>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  timedata = timedata.append(get_trip_data(r, '20230620'))



Index(['trip_id', 'time', 'lat', 'lon', 'stop_id', 'ritnumber', 'sequence'], dtype='object')
Index(['trip_id', 'time', 'lat', 'lon', 'stop_id', 'ritnumber', 'sequence'], dtype='object')


<ipython-input-19-cb0d1c70705c>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  timedata = timedata.append(get_trip_data(r, '20230620'))
<ipython-input-19-cb0d1c70705c>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  timedata = timedata.append(get_trip_data(r, '20230620'))
<ipython-input-19-cb0d1c70705c>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  timedata = timedata.append(get_trip_data(r, '20230620'))


Index(['trip_id', 'time', 'lat', 'lon', 'stop_id', 'ritnumber', 'sequence'], dtype='object')


<ipython-input-19-cb0d1c70705c>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  timedata = timedata.append(get_trip_data(r, '20230620'))


In [ ]:
timedata

,trip_id,ritnumber,sequence,time,stop_id,lat,lon,stop_code,stop_name
0,168758612,4016,1,384,2511899,51.924990,4.468890,RTD,Rotterdam Centraal
1,168249727,4034,1,654,2511899,51.924990,4.468890,RTD,Rotterdam Centraal
2,168249791,4038,1,714,2511899,51.924990,4.468890,RTD,Rotterdam Centraal
3,168759127,4048,1,864,2511899,51.924990,4.468890,RTD,Rotterdam Centraal
4,168250032,4052,1,924,2511899,51.924990,4.468890,RTD,Rotterdam Centraal
...,...,...,...,...,...,...,...,...,...
270103,166111090,2985,30,1482,2512035,51.045526,5.856709,STD,Sittard
270104,166111090,2985,30,1483,2512035,51.030885,5.856582,STD,Sittard
270105,166111090,2985,30,1484,2512035,51.016196,5.858848,STD,Sittard
270106,166111090,2985,31,1485,2512035,51.001640,5.858680,STD,Sittard


In [ ]:
timedata

,trip_id,ritnumber,sequence,time,stop_id,lat,lon,stop_code,stop_name
0,168758612,4016,1,384,2511899,51.924990,4.468890,RTD,Rotterdam Centraal
1,168249727,4034,1,654,2511899,51.924990,4.468890,RTD,Rotterdam Centraal
2,168249791,4038,1,714,2511899,51.924990,4.468890,RTD,Rotterdam Centraal
3,168759127,4048,1,864,2511899,51.924990,4.468890,RTD,Rotterdam Centraal
4,168250032,4052,1,924,2511899,51.924990,4.468890,RTD,Rotterdam Centraal
...,...,...,...,...,...,...,...,...,...
270103,166111090,2985,30,1482,2512035,51.045526,5.856709,STD,Sittard
270104,166111090,2985,30,1483,2512035,51.030885,5.856582,STD,Sittard
270105,166111090,2985,30,1484,2512035,51.016196,5.858848,STD,Sittard
270106,166111090,2985,31,1485,2512035,51.001640,5.858680,STD,Sittard


In [ ]:
timedata = timedata.merge(druktedata[['ritnumber', 'departure',
                                      'classification', 'seats', 'operator']],
                          left_on=['ritnumber', 'stop_code'],
                          right_on=['ritnumber', 'departure'])

timedata.time = timedata.time.apply(lambda x: min2str(x))
timedata['passengers'] = ((timedata.classification - 1) *
                          timedata.seats * 0.33).astype(int)
timedata['timestamp'] = (pd.to_datetime(timedata['time']) -
                         pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
timedata['elevation'] = 0

In [ ]:
timedata

#trip_id:         Unique identifier for a trip on a specific day
#ritnumber:       Tripnumber, specifies a trip from the GTFS
#sequence:        A trip consists of several sections.
                 #A section is the part of a trip between two consecutive stops or the time spent stationary at a stop
#time:            Scheduled arrival or departure time at the stop
#stop_id:         Unique identifier for a stop on a route
#lat:             Latitude of the stop location
#lon:             Longitude of the stop location
#stop_code:       Unique identifier for the stop in the NDOV dataset
#stop_name:       Name of the stop
#departure:       Scheduled departure time of the train from the stop
#classification:  A crowdedness classification for the train at the time of departure at the last stop
                 #Is a value between 1 and 5.
#seats:           Total number of seats in the train
#operator:        Name of the operator running the train
#passengers:      A rough estimate of the number of passengers in the train
#timestamp:       Unix timestamp, record for every minute a trip is underway
                 #time between arrival at first station and departure at last station
#elevation:       Elevation of the stop location

,trip_id,ritnumber,sequence,time,stop_id,lat,lon,stop_code,stop_name,departure,classification,seats,operator,passengers,timestamp,elevation
0,168758612,4016,1,06:24,2511899,51.92499,4.46889,RTD,Rotterdam Centraal,RTD,2,600,NS,198,1687242240,0
1,168758612,4016,1,06:24,2511899,51.92499,4.46889,RTD,Rotterdam Centraal,RTD,2,600,NS,198,1687242240,0
2,168758612,4016,1,06:24,2511899,51.92499,4.46889,RTD,Rotterdam Centraal,RTD,2,480,NS,158,1687242240,0
3,168758612,4016,1,06:24,2511899,51.92499,4.46889,RTD,Rotterdam Centraal,RTD,2,600,NS,198,1687242240,0
4,168758612,4016,1,06:24,2511899,51.92499,4.46889,RTD,Rotterdam Centraal,RTD,2,600,NS,198,1687242240,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2162349,166111090,2985,31,00:46,2512035,51.00164,5.85868,STD,Sittard,STD,2,400,NS,132,1687221960,0
2162350,166111090,2985,31,00:46,2512035,51.00164,5.85868,STD,Sittard,STD,2,400,NS,132,1687221960,0
2162351,166111090,2985,31,00:46,2512035,51.00164,5.85868,STD,Sittard,STD,2,400,NS,132,1687221960,0
2162352,166111090,2985,31,00:46,2512035,51.00164,5.85868,STD,Sittard,STD,2,600,NS,198,1687221960,0


In [ ]:
timedata

,trip_id,ritnumber,sequence,time,stop_id,lat,lon,stop_code,stop_name,departure,classification,seats,operator,passengers,timestamp,elevation
0,168758612,4016,1,06:24,2511899,51.92499,4.46889,RTD,Rotterdam Centraal,RTD,2,600,NS,198,1687242240,0
1,168758612,4016,1,06:24,2511899,51.92499,4.46889,RTD,Rotterdam Centraal,RTD,2,600,NS,198,1687242240,0
2,168758612,4016,1,06:24,2511899,51.92499,4.46889,RTD,Rotterdam Centraal,RTD,2,480,NS,158,1687242240,0
3,168758612,4016,1,06:24,2511899,51.92499,4.46889,RTD,Rotterdam Centraal,RTD,2,600,NS,198,1687242240,0
4,168758612,4016,1,06:24,2511899,51.92499,4.46889,RTD,Rotterdam Centraal,RTD,2,600,NS,198,1687242240,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2162349,166111090,2985,31,00:46,2512035,51.00164,5.85868,STD,Sittard,STD,2,400,NS,132,1687221960,0
2162350,166111090,2985,31,00:46,2512035,51.00164,5.85868,STD,Sittard,STD,2,400,NS,132,1687221960,0
2162351,166111090,2985,31,00:46,2512035,51.00164,5.85868,STD,Sittard,STD,2,400,NS,132,1687221960,0
2162352,166111090,2985,31,00:46,2512035,51.00164,5.85868,STD,Sittard,STD,2,600,NS,198,1687221960,0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Bio_Inspired_Project/Data

/content/drive/MyDrive/Bio_Inspired_Project/Data


In [ ]:
timedata.to_csv("/content/drive/MyDrive/Bio_Inspired_Project/Data/data2023_06.csv", index=False)

Swarm intelligence for train routing: The goal of this scenario would be to optimize the routing of trains based on the current and predicted crowdedness levels. One possible approach could be to use swarm intelligence algorithms, such as ant colony optimization or particle swarm optimization, to find the optimal routing solution. The motivation for this scenario is to improve the efficiency and punctuality of train services, as well as to reduce overcrowding and improve passenger experience.

In [ ]:
df1= timedata[timedata.trip_id==timedata.trip_id.unique()[j]]
df1.drop_duplicates(inplace=True)

In [ ]:
df1.columns

In [ ]:
df1.time.unique()

In [ ]:
#df1['time'] = pd.to_datetime(df1['time'], format='%H:%M')
df1.set_index(['time', 'sequence'], inplace=True)
df1.sort_index(inplace=True)
#df1.set_index(['time', 'sequence'])

In [ ]:
df1

In [ ]:
df1.trip_id.unique(), df1.ritnumber.unique(), df1.classification.unique(), df1.seats.unique(), df1.operator.unique(), df1.passengers.unique(), df1.elevation.unique()
#trip_id	ritnumber	sequence	time	stop_id	lat	lon	stop_code	stop_name	departure	classification	seats	operator	passengers	timestamp	elevation

In [ ]:
df1.set_index('sequence')

In [ ]:
# detect duplicates
duplicates = df1.duplicated()

# count the number of duplicates
num_duplicates = duplicates.sum()

In [ ]:
num_duplicates

<h3>Optimize the trajectory of one train </h3>
use ACS, mold(fungus)
CALCULATE THE DISTANCE


<h3>Optimize trajectories of multiple trains simultanuously </h3>
Intersections, ...

